## Clustering the images using k-means

In [ ]:
filename = 'result-wse_flickr.jsonl'
filepath = '../tasks/03.image-crawl'

import os

filename = os.path.join(filepath, filename)
print filename

print os.path.exists(filename)

# load the data
# only get the data for query='bench'

lines = open(filename).readlines()

import json
items = [json.loads(line) for line in lines]
items = filter(lambda item: item.get('embeds') and item.get('query') == 'bird', items)
print len(items)

print items[0].keys()

urls = [item['url'] for item in items]
features = [item['embeds'] for item in items]

print len(urls)
print len(features)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import KMeans
from sklearn import datasets

np.random.seed(5)

from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

# convert data to nparray
data = scale(np.array(features))
print data.shape

n_samples, n_features = data.shape
k = 3

k_means = KMeans(init='k-means++', n_clusters=k, n_init=10)
k_means.fit(data)


In [ ]:
clustered_urls = [[], [], []]

print clustered_urls

it = np.nditer(k_means.labels_, flags=['f_index'])

while not it.finished:
    #print it.index, it[0], urls[it.index]
    clustered_urls[it[0]].append(urls[it.index])
    #print clustered_urls
    it.iternext()

for idx in range(k):
    print 'count for %d: %d' % (idx, len(clustered_urls[idx]))

print clustered_urls[0][:10]
print
print clustered_urls[1][:10]


In [ ]:
# plow multiple images

"""
plot image matrix
"""

# plot 3x3 picture matrix

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import requests
import StringIO
from matplotlib.pyplot import figure, show, axes, sci
from PIL import Image


def thumbnail(img):
    """ generate 250x250 square thumbnail """
    
    THUMB_SIZE = 250, 250
    width, height = img.size

    if width > height:
        delta = width - height
        left = int(delta/2)
        upper = 0
        right = height + left
        lower = height
    else:
        delta = height - width
        left = 0
        upper = int(delta/2)
        right = width
        lower = width + upper

    img = img.crop((left, upper, right, lower))
    img.thumbnail(THUMB_SIZE, Image.ANTIALIAS)
    
    return img


def plot_3x3(images, title = ''):
    """
    given 9 images, plot them in 3x3 matrix
    """

    plt.figure(figsize=(10,10))
    
    Nr = 3
    Nc = 3
    i = 0
    
    for image in images:
        i += 1
        img_io = requests.get(image)
        plt.subplot(Nr, Nc, i)
        
        image = Image.open(StringIO.StringIO(img_io.content))
        image = thumbnail(image)
        
        #img = io.imread(StringIO.StringIO(img_io.content))
        plt.imshow(image)
    
    plt.show()


In [ ]:
# show the top 9 images for each cluster

import skimage.io as io
%matplotlib inline

for cat in range(3):
    img_urls = clustered_urls[cat][:9]
    plot_3x3(img_urls, title = '9 samples for cluster %d' % cat)
    print '---------------------'


In [ ]:
# get the nearest neighbour pictures

import numpy
from scipy.spatial import distance

def sim_search_k_images(images_features, search_feature):
    """
    images_features is an array of the features
    search_feature is the feature to be searched for nearest neighbour
    
    use distance measure to get the k nearest neighbours
    and return the indices of the samples
    """

    idx = 0
    results = []
    for feature in images_features:
        
        if numpy.array_equal(feature, search_feature):
            idx += 1
            continue
        
        dst = distance.euclidean(search_feature, feature)
        results.append((idx, dst))
        idx += 1
        
    top_results = sorted(results, key=lambda x: x[1])[:9]
    top_ids = [image[0] for image in top_results]
    return top_ids


In [ ]:
ids = sim_search_k_images(data, data[56])
print ids

img_urls = [urls[id] for id in ids]
print img_urls

plot_3x3(img_urls)